# 0.package install

In [1]:
# !pip install torch torchvision transformers pillow

# 1. captioning generation
1) CLIP으로 이미지에서 특징을 추출합니다.  
2) 추출된 이미지 특징을 GPT-2 모델에 입력합니다.  
3) 생성된 텍스트를 확인합니다.  


---
앞으로 구현할 것  
4) TTS 모델을 통해 생성된 텍스트를 음성으로 변환해줍니다.


In [8]:
import torch
from torchvision import transforms
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, GPT2Tokenizer, GPT2LMHeadModel
import requests

# CLIP 모델 및 프로세서 로드
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# GPT-2 모델 및 토크나이저 로드
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 이미지 로드 및 전처리
def load_image(image_path):
    # 이미지 로드 : 원하는 이미지로 바꿀 수 있어요
    url = "http://images.cocodataset.org/val2017/000000039769.jpg"
    image = Image. open(requests.get(url, stream=True).raw)
    image
    # image = Image.open(image_path)
    return image

# 이미지에서 특징 추출
def extract_image_features(image_path):
    image = load_image(image_path)
    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
    return features

# 이미지 특징을 기반으로 캡션 생성
def generate_caption_from_image(image_path):
    # 이미지에서 특징 추출
    image_features = extract_image_features(image_path)

    # GPT-2 모델을 사용하여 캡션 생성
    # 특징을 텍스트로 변환하기 위해 랜덤한 시작 문장을 사용합니다.
    embedding_dim = gpt2_model.config.n_embd  # GPT-2의 임베딩 차원 (768)
    linear_layer = torch.nn.Linear(image_features.size(-1), embedding_dim) #512 to 768

    # 임베딩 차원 변환
    transformed_embedding = linear_layer(image_features)


    # GPT-2 모델로 캡션 생성
    with torch.no_grad():
        outputs = gpt2_model.generate(
            inputs_embeds=transformed_embedding.unsqueeze(1),  # 임베딩 입력
            max_length=30,  # 최대 30 토큰 생성
            num_return_sequences=1,
            num_beams=5,  # 빔 서치로 더 나은 결과 생성
            early_stopping=True,
            pad_token_id=gpt2_tokenizer.pad_token_id  # Pad token 설정
        )
    generated_caption = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_caption

# 예시 이미지 경로
image_path = "path/to/your/image.jpg"  # 사용할 이미지 경로로 수정

# 캡션 생성
generated_caption = generate_caption_from_image(image_path)
print("Generated Caption:", generated_caption)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Caption: .

"I think it's a good idea," he said. "I think it's a good idea. I think it's a


In [4]:
image_features = extract_image_features(image_path)
image_features

tensor([[ 4.5713e-01, -4.8640e-01, -1.9568e-01, -5.6975e-01, -2.5177e-01,
         -2.8290e-01,  8.3081e-02,  5.7664e-01,  6.8331e-02, -4.4531e-01,
          7.0375e-02,  5.5453e-01,  1.9674e-01, -2.0233e-01, -1.9660e-01,
         -3.7810e-01, -6.6563e-02,  3.7217e-01,  4.9867e-01, -1.2863e-01,
          1.6052e-01, -3.5977e-01, -3.1558e-01, -5.1116e-01,  3.3640e-01,
         -5.6602e-01,  3.7456e-02,  7.7821e-02,  1.0731e-01,  2.0796e-01,
         -4.7783e-02, -8.8366e-02,  4.3023e-01, -2.0209e-01, -8.0026e-01,
         -1.1896e-02,  7.8789e-01,  4.3005e-02,  1.8673e-01,  8.1532e-02,
          2.1467e-01, -6.2337e-02,  4.4497e-01,  3.6166e-01, -1.2397e-01,
         -2.8634e-01,  1.0754e-01,  6.2397e-01, -3.7212e-01, -1.3557e-02,
         -1.6085e+00, -6.1973e-02,  1.8597e-01,  8.0334e-02, -1.2718e-01,
         -7.3099e-01, -4.3036e-01, -3.0935e-02, -1.8170e-01, -4.8850e-03,
          8.8081e-01, -1.5372e-01,  5.2346e-02, -6.4881e-02,  2.0808e-01,
          1.8223e-02,  8.9313e-02, -1.

In [5]:
image_features.shape

torch.Size([1, 512])

In [6]:
gpt2_model.config.n_embd 

768

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image. open(requests.get(url, stream=True).raw)
image